## 이 실습파일에서는 다음 high-level API를 사용하여 forward Propagation을 합니다.

- <span style = 'font-size:1.2em;line-height:1.5em'><b>torchvision.datasets: </b> pytorch에서 몇가지 sample data를 제공 (MNIST, Fashion-MNIST, CIFAR10, ...)</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>torchvision.transforms: </b>모델을 데이터에 입력하기 전에 전처리 하는 함수들을 제공</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>torch.utils.data.DataLoader: </b>Mini-batch data를 자동으로 생성하는 역할</span>

In [1]:
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader


### 1. Dataset 및 DataLoader생성하기

In [2]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
# (The MNIST dataset is also provided in torchvision)
# (Check whether the directory(data_path) exists. If not, create a new directory with that name.)
data_path = 'data2'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
# data transform method
# Following example: Transform the numpy arry data to torch tensor
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
# create a dataset (by torchvision.datasets)
trn_dset = datasets.MNIST(root=data_path, 
                          train=True, 
                          transform=transform, 
                          download=True)
tst_dset = datasets.MNIST(root=data_path, 
                          train=False, 
                          transform=transform, 
                          download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>DataLoader생성하기(train, test data 모두)</span>

In [5]:
373*160

59680

In [23]:
BATCH_SIZE = 10000
trn_loader = DataLoader(dataset=trn_dset, # 학습데이터로 mini-batch생성기를 만들거야
                        batch_size=BATCH_SIZE, # mini-batch의 크기는 BATCH_SIZE이고
                        shuffle=False, # 데이터의 순서를 섞어서 만들어줘
                        drop_last=False) # MIni-batch를 만들고 남는 데이터는 버려줘

# tst_loader = DataLoader(dataset=tst_dset, # 시험데이터로 mini-batch생성기를 만들거야
#                         batch_size=BATCH_SIZE, # mini-batch의 크기는 BATCH_SIZE이고
#                         shuffle=False, # 데이터의 순서를 섞지말고 만들어줘
#                         drop_last=False) # MIni-batch를 만들고 남는 데이터도 활용해줘

## 2. Model을 만드는 class 생성하기

### (1) 네트워크 구조 짜기
- <span style = 'font-size:1.2em;line-height:1.5em'>Input(784 dim) -> 1st hidden layer (50 dim) -> 2nd hidden layer (100 dim) -> output(10 dim)</span>
- <span style = 'font-size:1.2em;line-height:1.5em'>bias term 존재</span>
- <span style = 'font-size:1.2em;line-height:1.5em'>hidden layer에는 sigmoid, output layer에는 softmax activation function</span>

### (2) Input Flow 짜기
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 1.</b> (데이터 수 * 784)크기의 행렬(X)을 네트워크에 입력</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 2.</b> 1st hidden layer를 통과.  h1 = Matmul(X, W1) + b1</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>W1: (784*50) 크기의 행렬, b1: (50, )크기의 행렬, h1: (데이터 수 * 50) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 3.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 4.</b> 2nd hidden layer를 통과. h2 = Matmul(h1, W2) + b2</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W2: (50*100) 크기의 행렬, b2 = (100, )크기의 행렬, h2: (데이터 수 * 100) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 5.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 6.</b> output layer를 통과. h3 = Matmul(h2, W3) + b3</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W3: (100*10) 크기의 행렬, b3 = (10, )크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 7.</b> Activation함수 통과 (softmax)</span>

In [24]:
class MyNet(nn.Module): # nn.Module을 상속받아서 MyNet이라는 class를 생성한다.
    def __init__(self, in_dim, h1_dim, h2_dim, out_dim):
        super(MyNet, self).__init__() # 요건 습관처럼 쓰세요.
        
        # (1) 네트워크 구조 짜기 (with 단위 모듈)
        self.fc1 = nn.Linear(in_dim, h1_dim) # input->1st hidden layer에 대한 FFNN
        self.fc2 = nn.Linear(h1_dim, h2_dim) # 1st->2nd hidden layer에 대한 FFNN
        self.fc3 = nn.Linear(h2_dim, out_dim) # 2nd->output layer에 대한 FFNN
        
    def forward(self, x):
        # (2) Input Flow 짜기
        h1 = self.fc1(x) # step2
        h1 = F.sigmoid(h1) # step3
        h2 = self.fc2(h1) # step4
        h2 = F.sigmoid(h2) # step5
        h3 = self.fc3(h2) # step6
        out = F.softmax(h3, dim=1) # step7
        
        return out

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
device

device(type='cpu')

### 3. MyNet 클래스로부터 객체(object) 생성하기


In [27]:
net = MyNet(in_dim=784, # 입력하는 데이터의 차원 (784)
            h1_dim=50, # 1st hidden layer의 neuron 수 (50)
            h2_dim=100, # 2nd hidden layer의 neuron 수 (100)
            out_dim=10) # data의 class 수 = output layer의 neuron 수(10)
net = net.to(device) # 'gpu' 혹은 'cpu'에 network를 올려놓읍시다.

In [28]:
net

MyNet(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

- <span style = 'font-size:1.2em;line-height:1.5em'><b>Appendix: </b>기존에 학습한 모델을 net의 parameter들에 덮어씌우기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>(보통 이런식으로 안합니다만, 여기서는 그냥 그러려니 합시다.)</span>

In [29]:
fpath = 'sample_weights/sample_weight.pkl'
with open(fpath, 'rb') as f:
    network = pickle.load(f)

In [30]:
net.fc1.weight = nn.Parameter(torch.Tensor(network['W1'].T).to(device))
net.fc2.weight = nn.Parameter(torch.Tensor(network['W2'].T).to(device))
net.fc3.weight = nn.Parameter(torch.Tensor(network['W3'].T).to(device))
net.fc1.bias = nn.Parameter(torch.Tensor(network['b1']).to(device))
net.fc2.bias = nn.Parameter(torch.Tensor(network['b2']).to(device))
net.fc3.bias = nn.Parameter(torch.Tensor(network['b3']).to(device))

### 4. DataLoader로 Minibatch data를 생성하고 예측 결과를 누적하기

In [31]:
for i, (x_trn, y_trn) in enumerate(trn_loader):
    print(x_trn.view(-1, 784).shape, y_trn.shape)
    y_pred_score = net(x_trn.view(-1,784))
    y_pred_label = torch.argmax(y_pred_score, dim=1)
    y_pred_label = y_pred_label.detach().cpu().numpy()
    y_real_label = y_trn.detach().cpu().numpy()
    print(y_real_label)
    if i == 0:
        break

torch.Size([10000, 784]) torch.Size([10000])
[5 0 4 ... 6 9 7]


In [32]:
trn_pred_results = []
trn_real_results = []
for i, (x_trn, y_trn) in enumerate(trn_loader):
    x_trn = x_trn.view(-1,784).to(device)
    y_trn = y_trn.to(device)
    y_pred_score = net(x_trn)
    y_pred_label = torch.argmax(y_pred_score, dim=1)
    y_pred_label = y_pred_label.detach().cpu().numpy()
    y_real_label = y_trn.detach().cpu().numpy()
    trn_pred_results.extend(y_pred_label)
    trn_real_results.extend(y_real_label)
    
trn_pred_results = np.array(trn_pred_results)
trn_real_results = np.array(trn_real_results)
print(f'미리 학습한 모델의 정확도는 {100*np.sum(trn_pred_results==trn_real_results) / len(trn_real_results):.2f}%입니다.')

미리 학습한 모델의 정확도는 93.58%입니다.


- <span style = 'font-size:1.2em;line-height:1.5em'>이제까지는 결과가 계속 92.52%였는데, 여기서는 93.58%??</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>x data가 조금 다릅니다. 이전까지는 '밑바닥부터 시작하는 딥러닝'에서 제공하는 MNIST data, 여기서는 pytorch에서 제공하는 mnist data</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>원칙적으로는 같아야 하지만, 조금 다른게 있는듯. (중요한 건 아니니까 '그런 상황이 있구나' 하고 생각하시면 됩니다.</span>